In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib.pylab as pl
from fiona.crs import from_epsg
import fiona
from matplotlib import colors
import shapely
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap    
%matplotlib inline
pd.set_option('display.height',1000)
pd.set_option('display.max_row',500)
pd.set_option('display.max_columns',500)
pd.set_option("display.width", 1000)

height has been deprecated.



In [2]:
wasum = pd.DataFrame.from_csv('Census_Data/Water_District_Income_summary.csv')

In [3]:
bill = pd.DataFrame.from_csv('Census_Data/bill_15ccf_residential.csv')

In [5]:
bill.shape

(165, 98)

In [9]:
bill.columns.tolist()
bill.head()

,fuzzy_match,usage_ccf,cust_class,usage_month,days_in_period,usage_year,hhsize,meter_size,usage_zone,landscape_area,et_amount,wrap_customer,irr_area,carw_customer,season,tax_exemption,lot_size_group,temperature_zone,pressure_zone,water_font,city_limits,water_type,rate_class,dwelling_units,elevation_zone,greater_than,usage_indoor_budget_ccf,meter_type,block,tariff_area,turbine_meter,senior,utility_id,utility_name,effective_date,bill_frequency,unit_type,bill_type,service_charge,flat_rate,commodity_charge,bill,percentFixed,tier_starts,tier_prices,X1,X2,X3,XR1,XR2,XR3,X4,XR4,tier_starts_drought,tier_prices_drought,cost_adjustment_charge,X5,XR5,monthly_plant_factor,weather_adjustment_factor,volumetric_conversion_factor,irrigation_efficiency,indoor,outdoor,budget,gpcd,utility_surcharge_price,utility_surcharge,Tier_2,Tier_3,elevation_rate,elevation_charge,landscape_factor,minimum_charge,capital_improvements_charge,plant_factor,flat_rate_drought,sanitation_charge,outside_city_service_price,outside_city_service_charge,X6,XR6,fixed_water_service,mwd_ready_to_serve,sdcwa_infrastructure,X7,XR7,flat_service_charge,capital_facilities_charge,carw_charge,safe_drinking_water_surcharge,wrap_surcharge,wrap_discount,fixed_sewer_charge,sewer_charge,drought_surcharge,reliability_charge,PWSID
1,Alameda County Water District,15,RESIDENTIAL_SINGLE,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,28,Alameda County Water District,2017-03-01,Bi-Monthly,ccf,Uniform,24.92,4.047,60.70,85.62,0.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA0110001
2,Apple Valley Ranchos Water Company,15,RESIDENTIAL_SINGLE,7,31,2017,4,"3/4""",1,2000,7,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,379,Apple Valley Ranchos Water Company,2017-01-01,monthly,ccf,Tiered,34.73,NaN,63.14,97.87,0.35,0\r\n12\r\n24,4.039\r\n4.677\r\n5.315,11.0,4.0,0.0,44.429,18.708,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA3600010
3,Arcadia City of,15,RESIDENTIAL_SINGLE,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,132,Arcadia City Of,2017-04-01,Bimonthly,ccf,Tiered,10.17,NaN,24.46,34.63,0.29,0\r\n23\r\n49\r\n67,1.54\r\n1.88\r\n2.13\r\n2.29,22.0,8.0,0.0,33.880,15.040,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA1910003
4,Arrowbear Park County Water District,15,RESIDENTIAL_SINGLE,7,31,2017,4,"3/4""",1,2000,7,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,0,Arrowbear Park County Water District,2016-12-19,Monthly,ccf,Tiered,27.50,NaN,58.50,86.00,0.32,0\r\n7,0\r\n6.5,6.0,9.0,NaN,0.000,58.500,NaN,NaN,NaN,0\r\n5\r\n7\r\n11\r\n21,0\r\n6.5\r\n7.1\r\n7.7\r\n8.3\r\n9.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA3610110
5,Bear State Water Works,15,RESIDENTIAL_SINGLE,7,31,2017,4,"3/4""",1,2000,7,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,0,Bear State Water Works,2017-08-15,Monthly,ccf,Tiered,4.17,NaN,33.50,37.67,0.11,0\r\n6\r\n12,0\r\n2.25\r\n5,5.0,6.0,4.0,0.000,13.500,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CA0900217


In [21]:
col = [
 'usage_month',
 'days_in_period',
 'usage_year',
 'hhsize',
 'meter_size',
 'usage_zone',
 'landscape_area',
 'et_amount',
 'wrap_customer',
 'irr_area',
 'carw_customer',
 'season',
 'tax_exemption',
 'lot_size_group',
 'temperature_zone',
 'pressure_zone',
 'water_font',
 'city_limits',
 'water_type',
 'rate_class',
 'dwelling_units',
 'elevation_zone',
 'greater_than',
 'usage_indoor_budget_ccf',
 'meter_type',
 'block',
 'tariff_area',
 'turbine_meter',
 'senior',
 'utility_id',
 'utility_name',
 'effective_date',
 'bill_frequency',
 'unit_type',
 'bill_type',
 'service_charge',
 'flat_rate',
 'commodity_charge',
 'bill',
 'percentFixed',
 'PWSID']

In [14]:
bill[col].head(15)

,usage_month,days_in_period,usage_year,hhsize,meter_size,usage_zone,landscape_area,et_amount,wrap_customer,irr_area,carw_customer,season,tax_exemption,lot_size_group,temperature_zone,pressure_zone,water_font,city_limits,water_type,rate_class,dwelling_units,elevation_zone,greater_than,usage_indoor_budget_ccf,meter_type,block,tariff_area,turbine_meter,senior,utility_id,utility_name,effective_date,bill_frequency,unit_type,bill_type,service_charge,flat_rate,commodity_charge,bill,percentFixed,PWSID
1,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,28,Alameda County Water District,2017-03-01,Bi-Monthly,ccf,Uniform,24.92,4.047,60.70,85.62,0.29,CA0110001
2,7,31,2017,4,"3/4""",1,2000,7,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,379,Apple Valley Ranchos Water Company,2017-01-01,monthly,ccf,Tiered,34.73,NaN,63.14,97.87,0.35,CA3600010
3,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,132,Arcadia City Of,2017-04-01,Bimonthly,ccf,Tiered,10.17,NaN,24.46,34.63,0.29,CA1910003
4,7,31,2017,4,"3/4""",1,2000,7,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,0,Arrowbear Park County Water District,2016-12-19,Monthly,ccf,Tiered,27.50,NaN,58.50,86.00,0.32,CA3610110
5,7,31,2017,4,"3/4""",1,2000,7,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,0,Bear State Water Works,2017-08-15,Monthly,ccf,Tiered,4.17,NaN,33.50,37.67,0.11,CA0900217
6,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,383,Bellflower-Somerset Mutual Water Company,2014-10-01,bimonthly,ccf,Uniform,14.18,1.750,26.25,40.42,0.35,CA1910013
7,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,239,Beverly Hills City of,2017-07-03,Bimonthly,ccf,Tiered,21.68,NaN,71.00,92.68,0.23,CA1910156
8,7,31,2017,4,"3/4""",1,2000,7,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,275,Brentwood City of,2016-07-01,monthly,ccf,Tiered,29.83,NaN,63.02,92.85,0.32,CA0710004
9,7,31,2017,4,"3/4""",1,2000,7,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,270,Burbank City of,2017-01-02,monthly,ccf,Tiered,12.29,NaN,18.85,56.48,0.22,CA1910179
10,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,345,Burlingame City Of,2017-01-01,Bi-Monthly,ccf,Tiered,36.36,NaN,164.32,200.68,0.18,CA4110003


In [20]:
wasum.head()

,report_pwsid,Residential_Production_17_tot_summer,Target_Production_17_tot_summer,Residential_Production_17_tot_winter,Target_Production_17_tot_winter,Residential_Production_17_tot_spring,Target_Production_17_tot_spring,Residential_Production_17_tot_autumn,Target_Production_17_tot_autumn,Residential_Production_17_mean_summer,Target_Production_17_mean_summer,Residential_Production_17_mean_winter,Target_Production_17_mean_winter,Residential_Production_17_mean_spring,Target_Production_17_mean_spring,Residential_Production_17_mean_autumn,Target_Production_17_mean_autumn,report_agency_name,report_date,report_days_in_month,report_population,report_percent_residential,report_production_calculated,report_irr_area_sf,report_eto,Residential_Production,Target_Production,PWSID,Medium,Medium_val,color,x_medium
0,CA0110001,2.958398e+09,3.535869e+09,1.680220e+09,2.095374e+09,2.144144e+09,3.024893e+09,2.192910e+09,2.611627e+09,9.861325e+08,1.178623e+09,5.600733e+08,6.984581e+08,7.147147e+08,1.008298e+09,7.309700e+08,8.705424e+08,Alameda County Water District,2016-09-15,30,349000,0.67,1.281000e+09,1.859994e+08,4.84460,8.582700e+08,1.022792e+09,CA0110001,"$100,000 to $124,999",0.597880,10.0,$100k to 124.9k
12,CA0110005,1.148137e+10,2.029063e+10,6.735003e+09,9.829061e+09,8.153923e+09,1.675889e+10,8.593873e+09,1.368521e+10,3.827124e+09,6.763542e+09,2.245001e+09,3.276354e+09,2.717974e+09,5.586298e+09,2.864624e+09,4.561736e+09,East Bay Municipal Utilities District,2016-09-15,30,1400000,0.61,5.636200e+09,1.460161e+09,4.78277,3.438082e+09,5.773873e+09,CA0110005,"$60,000 to $74,999",0.510576,8.0,$60k to 74.9k
24,CA0110006,7.549160e+08,2.720004e+09,6.054853e+08,1.177968e+09,6.056126e+08,2.164450e+09,7.658761e+08,1.732526e+09,2.516387e+08,9.066680e+08,2.018284e+08,3.926561e+08,2.018709e+08,7.214835e+08,2.552920e+08,5.775088e+08,Hayward City of,2016-09-15,30,158985,0.52,5.117281e+08,2.011291e+08,4.86696,2.660986e+08,7.478532e+08,CA0110006,"$60,000 to $74,999",0.563936,8.0,$60k to 74.9k
36,CA0110008,9.968365e+08,1.318484e+09,3.802997e+08,5.747972e+08,6.047579e+08,1.033640e+09,6.325109e+08,8.359171e+08,3.322788e+08,4.394948e+08,1.267666e+08,1.915991e+08,2.015860e+08,3.445466e+08,2.108370e+08,2.786390e+08,Pleasanton City of,2016-09-15,30,74850,0.63,4.200000e+08,8.801671e+07,5.38935,2.646000e+08,3.587815e+08,CA0110008,"$125,000 to $149,999",0.593168,11.0,$125k to 149.9k
48,CA0110009,6.050637e+08,9.004183e+08,4.541701e+08,5.109019e+08,4.092036e+08,7.523737e+08,5.640451e+08,6.421191e+08,2.016879e+08,3.001394e+08,1.513900e+08,1.703006e+08,1.364012e+08,2.507912e+08,1.880150e+08,2.140397e+08,Dublin San Ramon Services District,2016-09-15,30,83227,0.68,2.785240e+08,4.460355e+07,5.24245,1.893963e+08,2.533052e+08,CA0110009,"$125,000 to $149,999",0.583049,11.0,$125k to 149.9k


In [23]:
mix_wasum_bill = wasum.merge(bill[col], on ='PWSID', how= 'inner')

In [24]:
mix_wasum_bill.shape

(123, 72)

In [32]:
#import requests
import plotly.plotly as py
import plotly.graph_objs as go 
import copy

from datetime import datetime
import plotly.figure_factory as ff

col = []
hist_data = []
group_labels = []
rug_text_all = []


hist_data.append(bill.bill)
group_labels.append('Total Bill')
rug_text_all.append(bill.PWSID+bill.utility_name)


fig = ff.create_distplot(hist_data, group_labels, show_hist=False, rug_text = rug_text_all, bin_size=5)

fig['layout'].update(title='Calculated Total Bill Distribution (15 ccf - July 2017)',
                     showlegend=True,

                     plot_bgcolor='rgb(250,250,250)',
                    xaxis = dict(title = 'Total Bill (USD)',gridcolor='#FFFFFF',#gridwidth=2
                                ),
              yaxis = dict(title = 'Density', gridcolor='#FFFFFF',
                          ))
# Plot!
py.iplot(fig, filename='test')

In [17]:
listforcolors = [ u'$15,000 to $19,999',
  u'$20,000 to $24,999',
  u'$25,000 to $29,999',
  u'$30,000 to $34,999',
  u'$35,000 to $39,999',
  u'$40,000 to $44,999',
  u'$45,000 to $49,999',
  u'$50,000 to $59,999',
  u'$60,000 to $74,999',
  u'$75,000 to $99,999',
  u'$100,000 to $124,999',
  u'$125,000 to $149,999',
  u'$150,000 to $199,999',
  u'$200,000 or more',]

In [25]:
mix_wasum_bill.head()

,report_pwsid,Residential_Production_17_tot_summer,Target_Production_17_tot_summer,Residential_Production_17_tot_winter,Target_Production_17_tot_winter,Residential_Production_17_tot_spring,Target_Production_17_tot_spring,Residential_Production_17_tot_autumn,Target_Production_17_tot_autumn,Residential_Production_17_mean_summer,Target_Production_17_mean_summer,Residential_Production_17_mean_winter,Target_Production_17_mean_winter,Residential_Production_17_mean_spring,Target_Production_17_mean_spring,Residential_Production_17_mean_autumn,Target_Production_17_mean_autumn,report_agency_name,report_date,report_days_in_month,report_population,report_percent_residential,report_production_calculated,report_irr_area_sf,report_eto,Residential_Production,Target_Production,PWSID,Medium,Medium_val,color,x_medium,usage_month,days_in_period,usage_year,hhsize,meter_size,usage_zone,landscape_area,et_amount,wrap_customer,irr_area,carw_customer,season,tax_exemption,lot_size_group,temperature_zone,pressure_zone,water_font,city_limits,water_type,rate_class,dwelling_units,elevation_zone,greater_than,usage_indoor_budget_ccf,meter_type,block,tariff_area,turbine_meter,senior,utility_id,utility_name,effective_date,bill_frequency,unit_type,bill_type,service_charge,flat_rate,commodity_charge,bill,percentFixed
0,CA0110001,2.958398e+09,3.535869e+09,1.680220e+09,2.095374e+09,2.144144e+09,3.024893e+09,2.192910e+09,2.611627e+09,9.861325e+08,1.178623e+09,5.600733e+08,6.984581e+08,7.147147e+08,1.008298e+09,7.309700e+08,8.705424e+08,Alameda County Water District,2016-09-15,30,349000,0.67,1.281000e+09,1.859994e+08,4.84460,8.582700e+08,1.022792e+09,CA0110001,"$100,000 to $124,999",0.597880,10.0,$100k to 124.9k,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,28,Alameda County Water District,2017-03-01,Bi-Monthly,ccf,Uniform,24.92,4.047,60.70,85.62,0.29
1,CA0110005,1.148137e+10,2.029063e+10,6.735003e+09,9.829061e+09,8.153923e+09,1.675889e+10,8.593873e+09,1.368521e+10,3.827124e+09,6.763542e+09,2.245001e+09,3.276354e+09,2.717974e+09,5.586298e+09,2.864624e+09,4.561736e+09,East Bay Municipal Utilities District,2016-09-15,30,1400000,0.61,5.636200e+09,1.460161e+09,4.78277,3.438082e+09,5.773873e+09,CA0110005,"$60,000 to $74,999",0.510576,8.0,$60k to 74.9k,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,891,East Bay Municipal Utility District,2017-07-12,Bi-Monthly,ccf,Tiered,22.60,NaN,77.30,99.90,0.23
2,CA0110006,7.549160e+08,2.720004e+09,6.054853e+08,1.177968e+09,6.056126e+08,2.164450e+09,7.658761e+08,1.732526e+09,2.516387e+08,9.066680e+08,2.018284e+08,3.926561e+08,2.018709e+08,7.214835e+08,2.552920e+08,5.775088e+08,Hayward City of,2016-09-15,30,158985,0.52,5.117281e+08,2.011291e+08,4.86696,2.660986e+08,7.478532e+08,CA0110006,"$60,000 to $74,999",0.563936,8.0,$60k to 74.9k,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,1294,Hayward City of,2016-10-01,bimonthly,ccf,Tiered,10.88,NaN,104.91,115.79,0.09
3,CA0110008,9.968365e+08,1.318484e+09,3.802997e+08,5.747972e+08,6.047579e+08,1.033640e+09,6.325109e+08,8.359171e+08,3.322788e+08,4.394948e+08,1.267666e+08,1.915991e+08,2.015860e+08,3.445466e+08,2.108370e+08,2.786390e+08,Pleasanton City of,2016-09-15,30,74850,0.63,4.200000e+08,8.801671e+07,5.38935,2.646000e+08,3.587815e+08,CA0110008,"$125,000 to $149,999",0.593168,11.0,$125k to 149.9k,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,403,Pleasanton City of,2017-01-15,bimonthly,ccf,Tiered,13.54,NaN,57.93,71.47,0.19
4,CA0110009,6.050637e+08,9.004183e+08,4.541701e+08,5.109019e+08,4.092036e+08,7.523737e+08,5.640451e+08,6.421191e+08,2.016879e+08,3.001394e+08,1.513900e+08,1.703006e+08,1.364012e+08,2.507912e+08,1.880150e+08,2.140397e+

In [33]:
data = []

for i in listforcolors:
#i=  listforcolors[5]
    trace_actual = go.Scatter(x= mix_wasum_bill[mix_wasum_bill['Medium']==i]['x_medium'],
                            y=mix_wasum_bill[mix_wasum_bill['Medium']==i].bill,
                            visible = True,
                            text= mix_wasum_bill[mix_wasum_bill['Medium']==i]['PWSID'] +' - '+ mix_wasum_bill[mix_wasum_bill['Medium']==i]['report_agency_name'],
                            name='Water Bill & Household Median Income',
                            marker=dict(size=16,
                                        cmax=13,
                                        cmin=0,
                                        color= mix_wasum_bill[mix_wasum_bill['Medium']==i].color.tolist(),
                                        line= dict(width=1),
                                        colorscale='Jet',),
                            mode='markers',
                            opacity = .55,)
    data.append(trace_actual)


                
        
layout = dict(title='Calculated Total Bill & Household Median Income. (15 ccf - July 2017)', showlegend=False,
              #width=900,
              #height=700,

              hovermode ='closest',
              yaxis=dict(
                        #range=[0, 28000000000],
                        title='Bill (USD)',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
                        ),), 
              xaxis=dict(
                        #range=[0, 28000000000],
                        title='Median Household Income',
                        
                        #tickvals = range(0,13),
                        #ticktext = listforcolors,
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
            
                        ),
                        ),
            )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='test')

In [38]:

col = []
hist_data = []
group_labels = []
rug_text_all = []


hist_data.append(bill.percentFixed)
group_labels.append('Fixed Price')
rug_text_all.append(bill.PWSID+bill.utility_name)


fig = ff.create_distplot(hist_data, group_labels, show_hist=False, rug_text = rug_text_all, bin_size=5)

fig['layout'].update(title='Water Fixed Price Distribution (15 ccf - July 2017)',
                     showlegend=True,

                     plot_bgcolor='rgb(250,250,250)',
                    xaxis = dict(title = 'Fixed Price Percentage',gridcolor='#FFFFFF',#gridwidth=2
                                ),
              yaxis = dict(title = 'Density', gridcolor='#FFFFFF',
                          ))
# Plot!
py.iplot(fig, filename='test')

In [35]:
data = []

for i in listforcolors:
#i=  listforcolors[5]
    trace_actual = go.Scatter(x= mix_wasum_bill[mix_wasum_bill['Medium']==i]['x_medium'],
                            y=mix_wasum_bill[mix_wasum_bill['Medium']==i].percentFixed,
                            visible = True,
                            text= mix_wasum_bill[mix_wasum_bill['Medium']==i]['PWSID'] +' - '+ mix_wasum_bill[mix_wasum_bill['Medium']==i]['report_agency_name'],
                            name='Water Bill & Household Median Income',
                            marker=dict(size=16,
                                        cmax=13,
                                        cmin=0,
                                        color= mix_wasum_bill[mix_wasum_bill['Medium']==i].color.tolist(),
                                        line= dict(width=1),
                                        colorscale='Jet',),
                            mode='markers',
                            opacity = .55,)
    data.append(trace_actual)


                
        
layout = dict(title='Percentage Fixed Price Bill & Household Median Income. (15 ccf - July 2017)', showlegend=False,
              #width=900,
              #height=700,

              hovermode ='closest',
              yaxis=dict(
                        #range=[0, 28000000000],
                        title='Percentage Fixed Price Bill',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
                        ),), 
              xaxis=dict(
                        #range=[0, 28000000000],
                        title='Median Household Income',
                        
                        #tickvals = range(0,13),
                        #ticktext = listforcolors,
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
            
                        ),
                        ),
            )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='test')

In [ ]:
mix_wasum_bill.Residential_Production_17_mean_summer

In [42]:
data=[]
trace_actual = go.Scatter(x=mix_wasum_bill.Residential_Production_17_mean_summer,
                            y=mix_wasum_bill.bill,
                            visible=True,
                            text= mix_wasum_bill['PWSID'] + ' - '+ mix_wasum_bill['report_agency_name'],
                            #name = color_s[s][0],
                            marker=dict(size=16,
                                        #color = color_s[s][1],
                                        line= dict(width=1),
                                       ),
                            opacity=.65,
                            #legendgroup = s,
                            #showlegend = show_leg,

                            mode='markers')
data.append(trace_actual)
                
layout = dict(title='Mean Water Performance by Season', showlegend=True,
              #shapes = line_1,
              width=900,
              height=700,
              yaxis=dict(
                        autorange = True,
                        #range=[0, 28000000000],
                        title='Bill (USD)',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
                        ),), 
              xaxis=dict(
                        autorange = True,
                        #range=[0, 28000000000],
                        title='Real Total Gallons',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
            
                        ),
                        ),
              legend = dict(orientation = 'v', 
                           ),
             )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='test')

In [53]:
li_name = ['bill', 'percentFixed']
colors = {'bill':['blue', 'Bill (USD)','y'], 'percentFixed':['orange','Fixed Price Bill (%)','y2']}

In [82]:
for i in li_name:
    print colors[i][2]

y
y2


In [111]:
data=[]
visshow=True
for i in li_name:
    trace_actual = go.Scatter(x=mix_wasum_bill.Residential_Production_17_mean_summer,
                                y=mix_wasum_bill[i],
                                visible=visshow,
                                text= mix_wasum_bill['PWSID'] + ' - '+ mix_wasum_bill['report_agency_name'],
                                name = colors[i][1],
                                marker=dict(size=16,
                                            color = colors[i][0],
                                            line= dict(width=1),
                                           ),
                                opacity=.65,
                                mode='markers')
    data.append(trace_actual)
    visshow=False
    
menu = []
vis = []
other=[]
vis.append(False)
other.append(True)

vis.append(True)
other.append(False)
print vis
print other
menu.append(dict(label = 'Bill (USD)',
                 method = 'update',
                 args = [{'visible': other,'yaxis':'y'}]
                ))

menu.append(dict(label = 'Fixed Price Bill (%)',
                 method = 'update',
                 args = [{'visible': vis,'yaxis':'y2'}]
                ))

updatemenus2 = list([
                    dict(active=0,
                         buttons=list(menu)
                    )
                ])
           
layout = dict(title='Bill vs Water Consumption', showlegend=True,
              #shapes = line_1,
              width=900,
              height=700,
              #paper_bgcolor ='lightgrey', 
              plot_bgcolor='rgb(240,240,240)', 

              hovermode = 'closest',
              yaxis=dict(
                        visible = True,
                        title='Bill (USD)',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
                        ),), 
              
              yaxis2=dict(
                        visible = True,
                        title='Fixed Price Bill (%)',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
                        ),), 
              
              
              xaxis=dict(
                        autorange = True,
                        #range=[0, 28000000000],
                        title='Real Total Gallons',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
            
                        ),
                        ),

              updatemenus=updatemenus2,
             )

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='test')

[False, True]
[True, False]


In [144]:
data=[]
visshow=True
for i in li_name:
    trace_actual = go.Scatter(x=mix_wasum_bill.Residential_Production_17_mean_summer/mix_wasum_bill.report_population,
                                y=mix_wasum_bill[i],
                                visible=visshow,
                                text= mix_wasum_bill['PWSID'] + ' - '+ mix_wasum_bill['report_agency_name'],
                                name = colors[i][1],
                                marker=dict(size=16,
                                            color = colors[i][0],
                                            line= dict(width=1),
                                           ),
                                opacity=.55,
                                mode='markers')
    data.append(trace_actual)
    visshow=False
    
menu = []
vis = []
other=[]
vis.append(False)
other.append(True)

vis.append(True)
other.append(False)
print vis
print other
menu.append(dict(label = 'Total Bill (USD)',
                 method = 'update',
                 args = [{'visible': other,'yaxis':'y'}]
                ))

menu.append(dict(label = 'Fixed Price on Bill (%)',
                 method = 'update',
                 args = [{'visible': vis,'yaxis':'y2'}]
                ))

updatemenus2 = list([
                    dict(active=0,
                         buttons=list(menu),
                            x = 0.13,
                            xanchor = 'left',
                            y = 1.08,
                            yanchor = 'top',
                    )
                ])
           
layout = dict(title='Bill vs Sumer Mean Water Consumption/perCap' , showlegend=True,
              margin = dict(pad=-50, t=-5),
              #shapes = line_1,
              width=900,
              height=700,
              #paper_bgcolor ='lightgrey', 
              plot_bgcolor='rgb(240,240,240)', 

              hovermode = 'closest',
              yaxis=dict(
                        autorange = True,
                        visible = True,
                        title='Bill (USD)',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
                        ),), 
              
              yaxis2=dict(
                        autorange = True,
                        visible = True,
                        title='Fixed Price Bill (%)',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
                        ),), 
              
              
              xaxis=dict(
                        autorange = True,
                        #range=[0, 28000000000],
                        title='Real Total Gallons',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
            
                        ),
                        ),

              updatemenus=updatemenus2,
             )

annotations = list([
    dict(text='Price Variable:', x=-10, y=1.07, yref='paper', align='left', showarrow=False)
])
layout['annotations'] = annotations

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='test')

[False, True]
[True, False]


In [147]:
mix_wasum_bill.head()

,report_pwsid,Residential_Production_17_tot_summer,Target_Production_17_tot_summer,Residential_Production_17_tot_winter,Target_Production_17_tot_winter,Residential_Production_17_tot_spring,Target_Production_17_tot_spring,Residential_Production_17_tot_autumn,Target_Production_17_tot_autumn,Residential_Production_17_mean_summer,Target_Production_17_mean_summer,Residential_Production_17_mean_winter,Target_Production_17_mean_winter,Residential_Production_17_mean_spring,Target_Production_17_mean_spring,Residential_Production_17_mean_autumn,Target_Production_17_mean_autumn,report_agency_name,report_date,report_days_in_month,report_population,report_percent_residential,report_production_calculated,report_irr_area_sf,report_eto,Residential_Production,Target_Production,PWSID,Medium,Medium_val,color,x_medium,usage_month,days_in_period,usage_year,hhsize,meter_size,usage_zone,landscape_area,et_amount,wrap_customer,irr_area,carw_customer,season,tax_exemption,lot_size_group,temperature_zone,pressure_zone,water_font,city_limits,water_type,rate_class,dwelling_units,elevation_zone,greater_than,usage_indoor_budget_ccf,meter_type,block,tariff_area,turbine_meter,senior,utility_id,utility_name,effective_date,bill_frequency,unit_type,bill_type,service_charge,flat_rate,commodity_charge,bill,percentFixed
0,CA0110001,2.958398e+09,3.535869e+09,1.680220e+09,2.095374e+09,2.144144e+09,3.024893e+09,2.192910e+09,2.611627e+09,9.861325e+08,1.178623e+09,5.600733e+08,6.984581e+08,7.147147e+08,1.008298e+09,7.309700e+08,8.705424e+08,Alameda County Water District,2016-09-15,30,349000,0.67,1.281000e+09,1.859994e+08,4.84460,8.582700e+08,1.022792e+09,CA0110001,"$100,000 to $124,999",0.597880,10.0,$100k to 124.9k,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,28,Alameda County Water District,2017-03-01,Bi-Monthly,ccf,Uniform,24.92,4.047,60.70,85.62,0.29
1,CA0110005,1.148137e+10,2.029063e+10,6.735003e+09,9.829061e+09,8.153923e+09,1.675889e+10,8.593873e+09,1.368521e+10,3.827124e+09,6.763542e+09,2.245001e+09,3.276354e+09,2.717974e+09,5.586298e+09,2.864624e+09,4.561736e+09,East Bay Municipal Utilities District,2016-09-15,30,1400000,0.61,5.636200e+09,1.460161e+09,4.78277,3.438082e+09,5.773873e+09,CA0110005,"$60,000 to $74,999",0.510576,8.0,$60k to 74.9k,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,891,East Bay Municipal Utility District,2017-07-12,Bi-Monthly,ccf,Tiered,22.60,NaN,77.30,99.90,0.23
2,CA0110006,7.549160e+08,2.720004e+09,6.054853e+08,1.177968e+09,6.056126e+08,2.164450e+09,7.658761e+08,1.732526e+09,2.516387e+08,9.066680e+08,2.018284e+08,3.926561e+08,2.018709e+08,7.214835e+08,2.552920e+08,5.775088e+08,Hayward City of,2016-09-15,30,158985,0.52,5.117281e+08,2.011291e+08,4.86696,2.660986e+08,7.478532e+08,CA0110006,"$60,000 to $74,999",0.563936,8.0,$60k to 74.9k,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,1294,Hayward City of,2016-10-01,bimonthly,ccf,Tiered,10.88,NaN,104.91,115.79,0.09
3,CA0110008,9.968365e+08,1.318484e+09,3.802997e+08,5.747972e+08,6.047579e+08,1.033640e+09,6.325109e+08,8.359171e+08,3.322788e+08,4.394948e+08,1.267666e+08,1.915991e+08,2.015860e+08,3.445466e+08,2.108370e+08,2.786390e+08,Pleasanton City of,2016-09-15,30,74850,0.63,4.200000e+08,8.801671e+07,5.38935,2.646000e+08,3.587815e+08,CA0110008,"$125,000 to $149,999",0.593168,11.0,$125k to 149.9k,7,62,2017,4,"3/4""",1,2000,14,No,2000,No,Summer,granted,3,Medium,1,city_delivered,inside_city,potable,C1,10,2,False,0.3,Turbine,1,1,No,no,403,Pleasanton City of,2017-01-15,bimonthly,ccf,Tiered,13.54,NaN,57.93,71.47,0.19
4,CA0110009,6.050637e+08,9.004183e+08,4.541701e+08,5.109019e+08,4.092036e+08,7.523737e+08,5.640451e+08,6.421191e+08,2.016879e+08,3.001394e+08,1.513900e+08,1.703006e+08,1.364012e+08,2.507912e+08,1.880150e+08,2.140397e+

In [149]:
li_name = ['bill', 'service_charge']
colors = {'bill':['blue', 'Bill (USD)','y'], 'service_charge':['orange','Fixed Price Bill','y2']}

In [1]:
data=[]
visshow=True
for i in li_name:
    trace_actual = go.Scatter(x=mix_wasum_bill.Residential_Production_17_mean_summer/mix_wasum_bill.report_population,
                                y=mix_wasum_bill[i],
                                visible=visshow,
                                text= mix_wasum_bill['PWSID'] + ' - '+ mix_wasum_bill['report_agency_name'],
                                name = colors[i][1],
                                marker=dict(size=16,
                                            color = colors[i][0],
                                            line= dict(width=1),
                                           ),
                                opacity=.55,
                                mode='markers')
    data.append(trace_actual)
    visshow=False
    
menu = []
vis = []
other=[]
vis.append(False)
other.append(True)

vis.append(True)
other.append(False)
print vis
print other
menu.append(dict(label = 'Total Bill (USD)',
                 method = 'update',
                 args = [{'visible': other,'yaxis':'y'}]
                ))

menu.append(dict(label = 'Fixed Price on Bill (%)',
                 method = 'update',
                 args = [{'visible': vis,'yaxis':'y2'}]
                ))

updatemenus2 = list([
                    dict(active=0,
                         buttons=list(menu),
                            x = 0.13,
                            xanchor = 'left',
                            y = 1.08,
                            yanchor = 'top',
                    )
                ])
           
layout = dict(title='Bill vs Sumer Mean Water Consumption/perCap' , showlegend=True,
              margin = dict(pad=-50, t=-5),
              #shapes = line_1,
              width=900,
              height=700,
              #paper_bgcolor ='lightgrey', 
              plot_bgcolor='rgb(240,240,240)', 

              hovermode = 'closest',
              yaxis=dict(
                        autorange = True,
                        visible = True,
                        title='Bill (USD)',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
                        ),), 
              
              yaxis2=dict(
                        autorange = True,
                        visible = True,
                        title='Fixed Price Bill (%)',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
                        ),), 
              
              
              xaxis=dict(
                        autorange = True,
                        #range=[0, 28000000000],
                        title='Real Total Gallons',
                        titlefont=dict(
                                        family='Arial, sans-serif',
                                        size=18,
                                        color='lightgrey'
            
                        ),
                        ),

              updatemenus=updatemenus2,
             )

annotations = list([
    dict(text='Price Variable:', x=-10, y=1.07, yref='paper', align='left', showarrow=False)
])
layout['annotations'] = annotations

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='test')

NameError: name 'li_name' is not defined

In [146]:
mix_wasum_bill.to_csv('Census_Data/WA_Income_Sum_Bill.csv')

In [145]:
75 + 12

87

In [ ]:
84 

In [ ]:
15 dias + 10sick +3per

In [ ]:
401k
